In [ ]:
import networkx as nx
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from networkx.algorithms import community as cm

### Lendo dataset Harry Potter

In [ ]:
atri = pd.read_csv('../data/harrypotter/hpattributes.txt', sep='\t')
ares = pd.read_csv('../data/harrypotter/hpbook2.txt', sep=' ', header=None)
nome = pd.read_csv('../data/harrypotter/hpnames.txt', sep='\t')


### CONCOR

In [ ]:
gpotter = nx.DiGraph()

In [ ]:
n = atri.shape[0]

In [ ]:
# config nodes
for k in range(n):
    gpotter.add_node(k,
                     nome=nome['name'][k],
                     ano=atri['schoolyear'][k],
                     gen=atri['gender'][k],
                     casa=atri['house'][k])


In [ ]:
# create network
for k in range(n):
    for m in range(n):
        if ares.values[k][m] == 1:
            gpotter.add_edge(k,m)

In [ ]:
nx.draw(gpotter,with_labels=True)

In [ ]:
# matrix de adjacencia
A = nx.adjacency_matrix(gpotter)

In [ ]:
# apagando os nos isolados 
isolados = []
for k in range(n):
    if np.sum(A[[k],:]) == 0:
        isolados.append(k)
        gpotter.remove_node(k)

print(len(isolados), 'nós isolados.')

In [ ]:
# matriz de adjacencia esparsa
A = nx.adjacency_matrix(gpotter)

In [ ]:
# matriz de adjacencia densa
A = np.array(A.todense(),dtype=float)

In [ ]:
#Algoritmo CONCOR
C   = A.copy()

while not np.allclose(np.abs(C),np.ones_like(C),atol=1e-10):
    C  = np.corrcoef(C)

C   = np.round(C)

In [ ]:
com1 = []
com2 = []

nos  = list(gpotter.nodes)

com1.append(nos[0])

for k in range(1,len(nos)):
    if C[0,k] > 0:
        com1.append(nos[k])
    else:
        com2.append(nos[k])

In [ ]:
print('Comunidade 1:')
for k in range(len(com1)):
    print(gpotter.nodes[com1[k]]['nome'])

In [ ]:
print('Comunidade 2:')
for k in range(len(com2)):
    print(gpotter.nodes[com2[k]]['nome'])

In [ ]:
plt.close('all')

fig, ax = plt.subplots(1,1,figsize=(15,10))

pos = nx.circular_layout(gpotter)

for k in range(len(com1)):
    pos[com1[k]][0] = 100*np.cos(2*np.pi*k/len(com1))
    pos[com1[k]][1] = 100*np.sin(2*np.pi*k/len(com1))

for k in range(len(com2)):
    pos[com2[k]][0] = 300*np.cos(2*np.pi*k/len(com2))
    pos[com2[k]][1] = 300*np.sin(2*np.pi*k/len(com2))

cores = []
for no in gpotter.nodes():
    if no in com1:
        cores.append('lightskyblue')
    else:
        cores.append('steelblue')

etiquetas = nx.get_node_attributes(gpotter, 'nome')

nx.draw_networkx(gpotter,
                 pos=pos,
                 edge_color='lightgray',
                 node_size = 500, 
                 labels=etiquetas,
                 font_size=10, 
                 node_color=cores)

plt.box(False)
#plt.xlim((-400,+400))
plt.show()

(module) community
Functions for computing and measuring community structure.

The community subpackage can be accessed by using networkx.community, then accessing the functions as attributes of community. 

For example:

In [ ]:
G = nx.barbell_graph(5, 1)
communities_generator = nx.community.girvan_newman(G)
top_level_communities = next(communities_generator)
next_level_communities = next(communities_generator)
print(sorted(map(sorted, top_level_communities)))
print(sorted(map(sorted, next_level_communities)))

In [ ]:
from networkx.algorithms import community as cm

mper = +np.inf
for ini in gpotter.nodes:
    com1 = set([ini])
    com2 = set(nos) - com1
    
    mgan = -1
    while mgan < 0:
        cv,pe = cm.partition_quality(gpotter,[com1,com2])
        ra = (1-cv)/(len(com1)*len(com2)) 
        mgan = +np.inf
        mno  = -1
        for no in com2:
            ncom1 = com1.union(set([no]))
            ncom2 = com2 - set([no])
            cv,pe = cm.partition_quality(gpotter,[ncom1,ncom2])
            nra   = (1-cv)/(len(ncom1)*len(ncom2))
            ng    = nra-ra
            if ng < mgan:
                mgan = ng
                mno  = no
        if mgan < 0:
            com1.add(mno)
            com2.remove(mno)
    cv,pe = cm.partition_quality(gpotter,[com1,com2])            
    ra = (1-cv)/(len(com1)*len(com2)) 
    if ra < mper:
        mper = ra
        mcom1 = com1.copy()
        mcom2 = com2.copy()

In [ ]:
com1 = list(mcom1)
com2 = list(mcom2)

In [ ]:
plt.close('all')

fig, ax = plt.subplots(1,1,figsize=(15,10))

pos = nx.circular_layout(gpotter)

for k in range(len(com1)):
    pos[com1[k]][0] = 300*np.cos(2*np.pi*k/len(com1))
    pos[com1[k]][1] = 300*np.sin(2*np.pi*k/len(com1))+700

for k in range(len(com2)):
    pos[com2[k]][0] = 300*np.cos(2*np.pi*k/len(com2))
    pos[com2[k]][1] = 300*np.sin(2*np.pi*k/len(com2))    

cores = []
for no in gpotter.nodes():
    if no in com1:
        cores.append('lightskyblue')
    else:
        cores.append('steelblue')

etiquetas = nx.get_node_attributes(gpotter, 'nome')

#plt.figure()
nx.draw_networkx(gpotter,pos=pos,edge_color='lightgray',
                 node_size = 500, labels=etiquetas,
                 font_size=10, node_color=cores)
plt.box(False)
plt.show()

In [ ]:
from networkx . algorithms import community as cm

com1, com2 = cm.kernighan_lin_bisection(gpotter.to_undirected(), max_iter =100)

In [ ]:
com1 = list(com1)
com2 = list(com2)

In [ ]:
plt.close('all')

fig, ax = plt.subplots(1,1,figsize=(15,10))

pos = nx.circular_layout(gpotter)

for k in range(len(com1)):
    pos[com1[k]][0] = 300*np.cos(2*np.pi*k/len(com1))
    pos[com1[k]][1] = 300*np.sin(2*np.pi*k/len(com1))+700

for k in range(len(com2)):
    pos[com2[k]][0] = 300*np.cos(2*np.pi*k/len(com2))
    pos[com2[k]][1] = 300*np.sin(2*np.pi*k/len(com2))    

cores = []
for no in gpotter.nodes():
    if no in com1:
        cores.append('lightskyblue')
    else:
        cores.append('steelblue')

etiquetas = nx.get_node_attributes(gpotter, 'nome')

#plt.figure()
nx.draw_networkx(gpotter,pos=pos,edge_color='lightgray',
                 node_size = 500, labels=etiquetas,
                 font_size=10, node_color=cores)
plt.box(False)
plt.show()

### Modularidade

In [ ]:
#%% MODULARIDADE: Clauset & Louvain

# coms = cm.community_louvain(gpotter)
comsl = cm.louvain_communities(gpotter)
comsc = cm.greedy_modularity_communities(gpotter,cutoff=len(comsl))


In [ ]:
print('Modularidade - Louvain: {:.3f}'.format(cm.modularity(gpotter,comsl)))
print('Modularidade - CNM    : {:.3f}'.format(cm.modularity(gpotter,comsc)))

In [ ]:
plt.close('all')

#fig, ax = plt.subplots(1,1,figsize=(15,10))

pos = nx.circular_layout(gpotter)

coresp = {0:'lightskyblue',
          1:'steelblue',
          2:'gray',
          3:'pink',
          4:'magenta',
          5:'violet',
          6: 'mediumorchid'}

coresl = []
for no in gpotter.nodes():
    for k in range(len(comsl)):
        if no in comsl[k]:
            coresl.append(coresp[k])
            break

coresc = []
for no in gpotter.nodes():
    for k in range(len(comsc)):
        if no in comsc[k]:
            coresc.append(coresp[k])
            break
    
etiquetas = nx.get_node_attributes(gpotter, 'nome')

In [ ]:
#plt.figure()

fig, ax = plt.subplots(1,1,figsize=(15,10))

nx.draw_networkx(gpotter,
                 pos=pos,
                 edge_color='lightgray',
                 node_size = 500,
                 labels=etiquetas,
                 font_size=10, 
                 node_color=coresl)

plt.box(False)
plt.show()

In [ ]:
#plt.figure()

fig, ax = plt.subplots(1,1,figsize=(15,10))

nx.draw_networkx(gpotter,
                 pos=pos,
                 edge_color='lightgray',
                 node_size = 500,
                 labels=etiquetas,
                 font_size=10, 
                 node_color=coresc)

plt.box(False)

plt.show()

### Modularidade espectral

### CONCOR

In [ ]:
gpotter = nx.DiGraph()

In [ ]:
n = atri.shape[0]

In [ ]:
# config nodes
for k in range(n):
    gpotter.add_node(k,
                     nome=nome['name'][k],
                     ano=atri['schoolyear'][k],
                     gen=atri['gender'][k],
                     casa=atri['house'][k])


In [ ]:
# create network
for k in range(n):
    for m in range(n):
        if ares.values[k][m] == 1:
            gpotter.add_edge(k,m)

In [ ]:
fig, ax = plt.subplots(1,1,figsize=(15,10))
nx.draw(gpotter,with_labels=True)
plt.show()

In [ ]:
# matrix de adjacencia
A = nx.adjacency_matrix(gpotter)

In [ ]:
# apagando os nos isolados 
isolados = []
for k in range(n):
    if np.sum(A[[k],:]) == 0:
        isolados.append(k)
        gpotter.remove_node(k)

print(len(isolados), 'nós isolados.')

In [ ]:
# matriz de adjacencia esparsa
A = nx.adjacency_matrix(gpotter)

In [ ]:
# matriz de adjacencia densa
A = np.array(A.todense(),dtype=float)

In [ ]:
# algoritmo CONCOR

C = A.copy()

while not np.allclose(np.abs(C),np.ones_like(C),atol=1e-10):
    C = np.corrcoef(C)

C = np.round(C)

In [ ]:
com1 = []
com2 = []

nos  = list(gpotter.nodes)

com1.append(nos[0])

for k in range(1,len(nos)):
    if C[0,k] > 0:
        com1.append(nos[k])
    else:
        com2.append(nos[k])

In [ ]:
print('Comunidade 1:')
for k in range(len(com1)):
    print(gpotter.nodes[com1[k]]['nome'])

In [ ]:
print('Comunidade 2:')
for k in range(len(com2)):
    print(gpotter.nodes[com2[k]]['nome'])

In [ ]:
plt.close('all')

fig, ax = plt.subplots(1,1,figsize=(15,10))

pos = nx.circular_layout(gpotter)

for k in range(len(com1)):
    pos[com1[k]][0] = 100*np.cos(2*np.pi*k/len(com1))
    pos[com1[k]][1] = 100*np.sin(2*np.pi*k/len(com1))

for k in range(len(com2)):
    pos[com2[k]][0] = 300*np.cos(2*np.pi*k/len(com2))
    pos[com2[k]][1] = 300*np.sin(2*np.pi*k/len(com2))

cores = []
for no in gpotter.nodes():
    if no in com1:
        cores.append('lightskyblue')
    else:
        cores.append('steelblue')

etiquetas = nx.get_node_attributes(gpotter, 'nome')

nx.draw_networkx(gpotter,
                 pos=pos,
                 edge_color='lightgray',
                 node_size = 500, 
                 labels=etiquetas,
                 font_size=10, 
                 node_color=cores)

plt.box(False)
#plt.xlim((-400,+400))
plt.show()

### Remoção de arestas

In [ ]:
from networkx . algorithms import community as cm

In [ ]:
com = cm.girvan_newman(gpotter)
com = list(com)

In [ ]:
for it in gpotter.nodes():
    print(it, ':', gpotter.nodes[it]['nome'])

In [ ]:
for it in com:
    print(it)

In [ ]:
fig, ax = plt.subplots(1,1,figsize=(15,10))

plt.plot([-3.90380859375,-3.90380859375],[0,-20],'b',linewidth=5)
plt.plot([0,-47.8076171875,40,40],[-20,-20,-20,-20],'b', linewidth=5)
plt.plot([40,40],[-20,-400],'b', linewidth=5)
plt.plot([-47.8076171875,-47.8076171875],[-20,-40],'b', linewidth=5)
plt.plot([-75.615234375,-20],[-40,-40],'b', linewidth=5)
plt.plot([-20,-20],[-40,-400],'b', linewidth=5)
plt.plot([-75.615234375,-75.615234375],[-40,-60],'b', linewidth=5)
plt.plot([-111.23046875,-40],[-60,-60],'b', linewidth=5)
plt.plot([-40,-40],[-60,-400],'b', linewidth=5)
plt.plot([-111.23046875,-111.23046875],[-60,-80],'b',linewidth=5)
plt.plot([-152.4609375,-70],[-80,-80],'b', linewidth=5)
# plt.plot([-80,-80],[-80,-100],'b', linewidth=5)
plt.plot([-152.4609375,-152.4609375],[-80,-100],'b', linewidth=5)
plt.plot([-179.921875,-125],[-100,-100],'b', linewidth=5)
# plt.plot([-80,-80],[-80,-100],'b', linewidth=5)
plt.plot([-179.921875,-179.921875],[-100,-120],'b', linewidth=5)
plt.plot([-125,-125],[-100,-220],'b', linewidth=5)
plt.plot([-199.84375,-160],[-120,-120],'b', linewidth=5)
plt.plot([-160,-160],[-120,-400],'b', linewidth=5)
plt.plot([-199.84375,-199.84375],[-120,-140],'b', linewidth=5)
plt.plot([-219.6875,-180],[-140,-140],'b', linewidth=5)
plt.plot([-180,-180],[-140,-400],'b', linewidth=5)
plt.plot([-219.6875,-219.6875],[-140,-160],'b', linewidth=5)
plt.plot([-239.375,-200],[-160,-160],'b', linewidth=5)
plt.plot([-200,-200],[-160,-400],'b', linewidth=5)
plt.plot([-239.375,-239.375],[-160,-180],'b', linewidth=5)
plt.plot([-258.75,-220],[-180,-180],'b', linewidth=5)
plt.plot([-220,-220],[-180,-400],'b', linewidth=5)
# plt.plot([-400,-400],[-180,-200],'b', linewidth=5)
# plt.plot([-120,-120],[-100,-220],'b', linewidth=5)
plt.plot([-140,-120],[-220,-220],'b', linewidth=5)
plt.plot([-140,-140],[-220,-400],'b', linewidth=5)
plt.plot([10,10],[-20,-240],'b', linewidth=5)
plt.plot([00,20],[-240,-240],'b', linewidth=5)
plt.plot([0,0],[-240,-400],'b', linewidth=5)
plt.plot([20,20],[-240,-400],'b', linewidth=5)
plt.plot([-258.75,-258.75],[-180,-260],'b', linewidth=5)
plt.plot([-277.5,-240],[-260,-260],'b', linewidth=5)   
plt.plot([-240,-240],[-260,-400],'b', linewidth=5)

plt.plot([-277.5,-277.5],[-260,-280],'b', linewidth=5)
plt.plot([-295,-260],[-280,-280],'b', linewidth=5)
plt.plot([-260,-260],[-280,-400],'b', linewidth=5)

plt.plot([-70,-70],[-80,-300],'b', linewidth=5)
plt.plot([-80,-60],[-300,-300],'b', linewidth=5)
plt.plot([-80,-80],[-300,-400],'b', linewidth=5)
plt.plot([-60,-60],[-300,-400],'b', linewidth=5)

plt.plot([-295,-295],[-280,-320],'b', linewidth=5)
plt.plot([-315,-280],[-320,-320],'b', linewidth=5)
plt.plot([-280,-280],[-320,-400],'b', linewidth=5)
plt.plot([-315,-315],[-320,-340],'b', linewidth=5)

plt.plot([-340,-320],[-360,-360],'b', linewidth=5)
plt.plot([-340,-340],[-360,-400],'b', linewidth=5)
plt.plot([-320,-320],[-360,-400],'b', linewidth=5)

    
plt.plot([-330,-330],[-340,-360],'b', linewidth=5)
plt.plot([-330,-300],[-340,-340],'b', linewidth=5)
plt.plot([-300,-300],[-340,-400],'b', linewidth=5)


plt.plot([-140,-110],[-220,-220],'b', linewidth=5)
plt.plot([-110,-110],[-220,-380],'b', linewidth=5)
plt.plot([-120,-100],[-380,-380],'b', linewidth=5)
plt.plot([-120,-120],[-380,-400],'b', linewidth=5)
plt.plot([-100,-100],[-380,-400],'b', linewidth=5)

plt.box(False)
plt.xticks([])
plt.yticks([])
plt.ylim(-580,0)
plt.text(-340,-580,gpotter.nodes[28]['nome'],rotation='vertical',horizontalalignment='center',fontsize=14)
plt.text(-320,-580,gpotter.nodes[44]['nome'],rotation='vertical',horizontalalignment='center',fontsize=14)

plt.text(-300,-580,gpotter.nodes[25]['nome'],rotation='vertical',horizontalalignment='center',fontsize=14)

plt.text(-280,-580,gpotter.nodes[24]['nome'],rotation='vertical',horizontalalignment='center',fontsize=14)

plt.text(-260,-580,gpotter.nodes[19]['nome'],rotation='vertical',horizontalalignment='center',fontsize=14)

plt.text(-240,-580,gpotter.nodes[18]['nome'],rotation='vertical',horizontalalignment='center',fontsize=14)

plt.text(-220,-580,gpotter.nodes[ 2]['nome'],rotation='vertical',horizontalalignment='center',fontsize=14)

plt.text(-200,-580,gpotter.nodes[ 1]['nome'],rotation='vertical',horizontalalignment='center',fontsize=14)

plt.text(-180,-580,gpotter.nodes[55]['nome'],rotation='vertical',horizontalalignment='center',fontsize=14)

plt.text(-160,-580,gpotter.nodes[32]['nome'],rotation='vertical',horizontalalignment='center',fontsize=14)

plt.text(-140,-580,gpotter.nodes[10]['nome'],rotation='vertical',horizontalalignment='center',fontsize=14)
plt.text(-120,-580,gpotter.nodes[43]['nome'],rotation='vertical',horizontalalignment='center',fontsize=14)
plt.text(-100,-580,gpotter.nodes[57]['nome'],rotation='vertical',horizontalalignment='center',fontsize=14)

plt.text(- 80,-580,gpotter.nodes[20]['nome'],rotation='vertical',horizontalalignment='center',fontsize=14)
plt.text(- 60,-580,gpotter.nodes[51]['nome'],rotation='vertical',horizontalalignment='center',fontsize=14)

plt.text(- 40,-580,gpotter.nodes[27]['nome'],rotation='vertical',horizontalalignment='center',fontsize=14)

plt.text(- 20,-580,gpotter.nodes[23]['nome'],rotation='vertical',horizontalalignment='center',fontsize=14)

plt.text(-  0,-580,gpotter.nodes[13]['nome'],rotation='vertical',horizontalalignment='center',fontsize=14)
plt.text(+ 20,-580,gpotter.nodes[38]['nome'],rotation='vertical',horizontalalignment='center',fontsize=14)

plt.text(+ 40,-580,gpotter.nodes[16]['nome'],rotation='vertical',horizontalalignment='center',fontsize=14)

plt.show()

### Agrupamento hierárquico

import scipy.cluster.hierarchy as hier

In [ ]:
import scipy.cluster.hierarchy as hier

In [ ]:
A = nx.adjacency_matrix(gpotter)

A = np.array(A.todense(), dtype = float)

S = np.copy(A.T)

In [ ]:
Z = hier.linkage(S, 'complete', metric='euclidean')

In [ ]:
nomes = [gpotter.nodes[n]['nome'] for n in gpotter.nodes]

In [ ]:
hier.set_link_color_palette(['b'])

In [ ]:
fig, ax = plt.subplots(1, 1, figsize = (15,10))

with plt.rc_context({'lines.linewidth': 5}):
    dn = hier.dendrogram(Z, 
                         color_threshold = np.inf,
                         leaf_rotation = 'vertical', 
                         leaf_font_size = 14, 
                         labels = nomes )
    
plt.tight_layout()
plt.box(False)
#_ = plt.yticks ()
plt.show()